# Limpieza de datos en 15 pasos para Sumaria 2004 - 2019 INEI (Perú)

In [1]:
import pandas as pd
import numpy as np
import pyreadstat
import os

# Cargar archivos

In [2]:
data_list = os.listdir('./data')
e = []
d = []
for x in data_list:
    print('./data/' + x)
    data, meta = pyreadstat.read_sav('./data/' + x)
    e.append(data)
    d.append(meta)

./data/sumaria-2004.sav
./data/sumaria-2005.sav
./data/sumaria-2006.sav
./data/sumaria-2007-2011.sav
./data/sumaria-2011-2015.sav
./data/sumaria-2015-2019.sav


#### Variables de interés

* AÑO|A_O_ : Año 
* UBIGEO : Ubicación geografica 
* INGHOG2D : Ingreso neto total
* INGHOG1D': Ingreso bruto total
* POBREZA : Pobreza  
* PERCEPHO : Total de perceptores de ingresos
* MIEPERHO : Total de miembros del hogar
* LINEA : Linea de pobreza total
* ESTRATO : Estrato geográfico
* FACTOR07 : Factor expansion ajustado a CPV 2007
* LD : Deflactor espacial


# Clear Data

In [3]:
# convierte todos los nombres de las columnas en mayúsculas
for x in e:
    x.columns = x.columns.str.upper()

In [4]:
# podemos ver las variables potencialmente útiles 
list_columns = ['AÑO|A_O_', 'UBIGEO', 'INGHOG1D', 'INGHOG2D', 'POBREZA', 'PERCEPHO',
                'MIEPERHO', 'LINEA', 'ESTRATO', 'FACTOR07', 'LD' ]
for x in e:
    for col in list_columns:
        print(x.columns[x.columns.str.contains(col)], x.iloc[0][0])    

Index(['AÑO'], dtype='object') 2004
Index(['UBIGEO'], dtype='object') 2004
Index(['INGHOG1D'], dtype='object') 2004
Index(['INGHOG2D'], dtype='object') 2004
Index(['POBREZA'], dtype='object') 2004
Index(['PERCEPHO'], dtype='object') 2004
Index(['MIEPERHO'], dtype='object') 2004
Index(['LINEA'], dtype='object') 2004
Index(['ESTRATO'], dtype='object') 2004
Index(['FACTOR07'], dtype='object') 2004
Index([], dtype='object') 2004
Index(['AÑO'], dtype='object') 2005
Index(['UBIGEO'], dtype='object') 2005
Index(['INGHOG1D'], dtype='object') 2005
Index(['INGHOG2D'], dtype='object') 2005
Index(['POBREZA'], dtype='object') 2005
Index(['PERCEPHO'], dtype='object') 2005
Index(['MIEPERHO'], dtype='object') 2005
Index(['LINEA'], dtype='object') 2005
Index(['ESTRATO'], dtype='object') 2005
Index(['FACTOR07'], dtype='object') 2005
Index([], dtype='object') 2005
Index(['AÑO'], dtype='object') 2006
Index(['UBIGEO'], dtype='object') 2006
Index(['INGHOG1D'], dtype='object') 2006
Index(['INGHOG2D'], dtype=

In [5]:
# concat desde el 2004 al 2006

list_columns = ['AÑO|A_O_|UBIGEO|INGHOG1D|INGHOG2D|POBREZA|PERCEPHO|MIEPERHO|LINEA|ESTRATO|FACTOR07|LD' ]
df_04_06 = pd.DataFrame()
for x in e[0:3]:
    
    for col in list_columns:
        y = x.loc[:,x.columns[x.columns.str.contains(col)]]
        y = y.rename(columns={'AÑO': 'AÑO|A_O_' })
    df_04_06 = pd.concat([df_04_06, y], axis = 0)
df_04_06 = df_04_06.dropna() 
df_04_06

,AÑO|A_O_,UBIGEO,ESTRATO,PERCEPHO,MIEPERHO,INGHOG2D,INGHOG1D,LINEA,POBREZA,FACTOR07
0,2004,010101,4.0,1.0,2.0,6442.551818,6442.551818,204.297028,3.0,130.410004
1,2004,010101,4.0,4.0,7.0,37598.200456,39502.388756,204.297028,3.0,130.410004
2,2004,010101,4.0,2.0,3.0,16512.070455,18258.401070,204.297028,3.0,130.410004
3,2004,010201,4.0,1.0,2.0,3217.789921,3217.789921,231.336853,2.0,98.389999
4,2004,010201,4.0,1.0,3.0,12221.894937,12221.894937,231.336853,3.0,98.389999
...,...,...,...,...,...,...,...,...,...,...
20572,2006,250103,7.0,2.0,5.0,8766.844097,8766.844097,171.714417,2.0,59.439999
20573,2006,250103,7.0,1.0,6.0,9202.902017,9202.902017,171.714417,1.0,59.439999
20574,2006,250103,7.0,1.0,3.0,4771.336854,4771.336854,171.714417,2.0,59.439999
20575,2006,250103,7.0,2.0,6.0,13695.818211,13695.818211,171.714417,2.0,59.439999


In [6]:
# Se borra la variable del año 2015, debido a que se repite en dos base de datos que se van concatenar

e[4] = e[4].loc[:, e[4].columns[~e[4].columns.str.contains('15')]]

In [7]:
# Se cambia de nombre a la variable debido a la confusión que puede generar

e[4] = e[4].rename(columns={'FACTOR07' : 'FACTOR07_15'})

In [8]:
# Elección de variables y concatenar los datos anualmente desde 2007 hasta el 2011 

columns = ['AÑO|A_O_', 'UBIGEO','ESTRATO','PERCEPHO','MIEPERHO','INGHOG2D', 'INGHOG1D','LINEA', 'POBREZA', 'FACTOR07', 'LD' ]
columns.remove('FACTOR07')
columns.remove('LD')
list_columns = ['AÑO|A_O_|UBIGEO|INGHOG1D|INGHOG2D|POBREZA|PERCEPHO|MIEPERHO|LINEA|ESTRATO|FACTOR07|LD' ]
df_07_11 = pd.DataFrame()

for x in e[3:4]:
    for col in list_columns:
        y = x.loc[:, x.columns[x.columns.str.contains(col)]]
        for i in range(7, 12):
            z = y.loc[ :, y.columns.str.contains('{}'.format(i))]
            z.columns = columns
            df_07_11 = pd.concat([df_07_11, z], axis=0)
df_07_11 = df_07_11.dropna()
df_07_11

,AÑO|A_O_,UBIGEO,ESTRATO,PERCEPHO,MIEPERHO,INGHOG2D,INGHOG1D,LINEA,POBREZA
1,2007,010101,4.0,3.0,4.0,30184.485080,31709.485926,203.656921,3.0
2,2007,010101,4.0,3.0,6.0,146356.726231,154767.511773,203.656921,3.0
3,2007,010101,4.0,2.0,2.0,7521.646561,7521.646561,203.656921,3.0
4,2007,010101,4.0,3.0,4.0,11644.447752,13595.653044,203.656921,3.0
5,2007,010101,4.0,5.0,6.0,39635.427690,41127.837017,203.656921,3.0
...,...,...,...,...,...,...,...,...,...
113980,2011,190101,3.0,2.0,5.0,13381.000000,13381.000000,238.358856,3.0
113983,2011,190101,3.0,2.0,3.0,19462.000000,18573.000000,238.358856,3.0
113987,2011,190101,3.0,2.0,4.0,70008.000000,59027.000000,238.358856,3.0
113992,2011,190101,3.0,3.0,5.0,88706.000000,82477.000000,238.358856,3.0


In [9]:
# Elección  de variables y concatenar los datos anualmente desde 2012 hasta el 2019 

columns = ['AÑO|A_O_', 'UBIGEO', 'INGHOG1D', 'INGHOG2D', 'POBREZA', 
           'PERCEPHO','MIEPERHO', 'LINEA', 'ESTRATO', 'FACTOR07', 'LD' ]
list_columns = ['AÑO|A_O_|UBIGEO|INGHOG1D|INGHOG2D|POBREZA|PERCEPHO|MIEPERHO|LINEA|ESTRATO|FACTOR07|LD' ]
df_12_19 = pd.DataFrame()
for x in e[4:]:
    
    for col in list_columns:
        y = x.loc[:,x.columns[x.columns.str.contains(col)]]
        for i in range(12, 20):
            if y.loc[ :, y.columns.str.contains('{}'.format(i))].columns.size == 11:
                z = y.loc[ :, y.columns.str.contains('{}'.format(i))]
                z.columns = columns
                df_12_19 = pd.concat([df_12_19, z], axis = 0)
df_12_19 = df_12_19.dropna()
df_12_19

,AÑO|A_O_,UBIGEO,INGHOG1D,INGHOG2D,POBREZA,PERCEPHO,MIEPERHO,LINEA,ESTRATO,FACTOR07,LD
0,2012,010101,4.0,144.103,1.0,1.000000,21809.425648,21809.425648,0.815806,245.544647,3.00000
1,2012,010101,4.0,144.103,4.0,7.000000,79845.938186,73907.938186,0.815806,245.544647,3.00000
2,2012,010101,4.0,144.103,3.0,5.000000,63911.000000,58993.000000,0.815806,245.544647,3.00000
3,2012,010101,4.0,144.103,3.0,5.000000,23035.000000,23035.000000,0.815806,245.544647,2.00000
4,2012,010101,4.0,144.103,5.0,5.000000,60096.000000,56774.000000,0.815806,245.544647,3.00000
...,...,...,...,...,...,...,...,...,...,...,...
181184,2019,180101,3.0,2.000,3.0,47931.878906,43273.878906,0.854033,345.210449,3.000000,80.25766
181185,2019,180101,3.0,2.000,3.0,28237.265625,28237.265625,0.854033,345.210449,3.000000,80.25766
181186,2019,180101,3.0,2.000,3.0,53529.687500,50113.687500,0.854033,345.210449,3.000000,80.25766
181187,2019,180101,3.0,4.000,6.0,117474.468750,105555.468750,0.854033,345.210449,3.000000,80.25766


# Concat general

En este paso se apila la base de datos concatenados anteriormente.

In [10]:
df_concat = pd.concat([df_04_06, df_07_11, df_12_19], axis=0)
df_concat

,AÑO|A_O_,UBIGEO,ESTRATO,PERCEPHO,MIEPERHO,INGHOG2D,INGHOG1D,LINEA,POBREZA,FACTOR07,LD
0,2004,010101,4.000000,1.000000,2.000000,6442.551818,6442.551818,204.297028,3.0,130.410004,NaN
1,2004,010101,4.000000,4.000000,7.000000,37598.200456,39502.388756,204.297028,3.0,130.410004,NaN
2,2004,010101,4.000000,2.000000,3.000000,16512.070455,18258.401070,204.297028,3.0,130.410004,NaN
3,2004,010201,4.000000,1.000000,2.000000,3217.789921,3217.789921,231.336853,2.0,98.389999,NaN
4,2004,010201,4.000000,1.000000,3.000000,12221.894937,12221.894937,231.336853,3.0,98.389999,NaN
...,...,...,...,...,...,...,...,...,...,...,...
181184,2019,180101,345.210449,47931.878906,43273.878906,2.000000,3.000000,0.854033,3.0,3.000000,80.25766
181185,2019,180101,345.210449,28237.265625,28237.265625,2.000000,3.000000,0.854033,3.0,3.000000,80.25766
181186,2019,180101,345.210449,53529.687500,50113.687500,2.000000,3.000000,0.854033,3.0,3.000000,80.25766
181187,2019,180101,345.210449,117474.468750,105555.468750,4.000000,3.000000,0.854033,6.0,3.000000,80.25766


El siguiente paso es identificar las regiones, para ello se hará uso de UBIGEO, el cual los dos primeros digidos indican la región, sin embargo, este aún se encuentran en números. Para lograr identificar las regiones relacionado a su código utilizamos la siguiente clasificación:

* 1 : Amazonas
* 2 : Áncash
* 3 :  Apurímac
* ...
* 25 : Ucayali

In [11]:
df_concat['region']=round(df_concat['UBIGEO'].astype(int)/10000)
df_concat['region'] = df_concat['region'].astype(int)
df_concat

,AÑO|A_O_,UBIGEO,ESTRATO,PERCEPHO,MIEPERHO,INGHOG2D,INGHOG1D,LINEA,POBREZA,FACTOR07,LD,region
0,2004,010101,4.000000,1.000000,2.000000,6442.551818,6442.551818,204.297028,3.0,130.410004,NaN,1
1,2004,010101,4.000000,4.000000,7.000000,37598.200456,39502.388756,204.297028,3.0,130.410004,NaN,1
2,2004,010101,4.000000,2.000000,3.000000,16512.070455,18258.401070,204.297028,3.0,130.410004,NaN,1
3,2004,010201,4.000000,1.000000,2.000000,3217.789921,3217.789921,231.336853,2.0,98.389999,NaN,1
4,2004,010201,4.000000,1.000000,3.000000,12221.894937,12221.894937,231.336853,3.0,98.389999,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
181184,2019,180101,345.210449,47931.878906,43273.878906,2.000000,3.000000,0.854033,3.0,3.000000,80.25766,18
181185,2019,180101,345.210449,28237.265625,28237.265625,2.000000,3.000000,0.854033,3.0,3.000000,80.25766,18
181186,2019,180101,345.210449,53529.687500,50113.687500,2.000000,3.000000,0.854033,3.0,3.000000,80.25766,18
181187,2019,180101,345.210449,117474.468750,105555.468750,4.000000,3.000000,0.854033,6.0,3.000000,80.25766,18


#### Carga archivos adicionales

Cargamos los datos de la clasificación por regiones

In [12]:
ubigeo = pd.read_csv('ubigeo_dep_peru_16.csv')
ubigeo = ubigeo.rename(columns={'id': 'region', 'name':'nombre'})
ubigeo.head()

,region,nombre
0,1,Amazonas
1,2,Áncash
2,3,Apurímac
3,4,Arequipa
4,5,Ayacucho


#### Merge

En este paso se idenfican las regiones

In [13]:
df_concat_region = pd.merge(df_concat, ubigeo, on='region')
df_concat_region.head()

,AÑO|A_O_,UBIGEO,ESTRATO,PERCEPHO,MIEPERHO,INGHOG2D,INGHOG1D,LINEA,POBREZA,FACTOR07,LD,region,nombre
0,2004,010101,4.0,1.0,2.0,6442.551818,6442.551818,204.297028,3.0,130.410004,NaN,1,Amazonas
1,2004,010101,4.0,4.0,7.0,37598.200456,39502.388756,204.297028,3.0,130.410004,NaN,1,Amazonas
2,2004,010101,4.0,2.0,3.0,16512.070455,18258.401070,204.297028,3.0,130.410004,NaN,1,Amazonas
3,2004,010201,4.0,1.0,2.0,3217.789921,3217.789921,231.336853,2.0,98.389999,NaN,1,Amazonas
4,2004,010201,4.0,1.0,3.0,12221.894937,12221.894937,231.336853,3.0,98.389999,NaN,1,Amazonas


In [14]:
# podemos ver la cantidad de observaciones por año

df_concat_region['AÑO|A_O_'].value_counts()

2018    37462
2016    35785
2017    34584
2019    34565
2015    32188
2014    30848
2013    30453
2012    25091
2011    24809
2007    22204
2009    21753
2008    21502
2010    21496
2006    20577
2005    19895
2004    19502
Name: AÑO|A_O_, dtype: int64

Por último exportamos la base de datos ya limpia

In [15]:
df_concat_region.to_csv('sumaria_04_19.csv')